In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from binance.client import Client
import ta

# Autenticación en la API de Binance
api_key = 'T4Bw573BSJCbaHscSo8jn37lE1SOoGsNircF8f7B061WIKBxNkP5nx68vvAev9uk'
api_secret = 'p7VN6HHKR6ZtGkfEeZC3FongJPkR2yr7AjPtPDC8IeaOkASdKZYCKyTuiIfGw57q'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.futures_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "13 year ago UTC")

# Creación de un dataframe con los datos
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                     'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 
                                     'taker_buy_quote_asset_volume', 'ignore'])

# Eliminación de las columnas que no se van a utilizar
data = data.drop(['timestamp', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 
                  'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)

# Conversión de los precios a float
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['close'] = data['close'].astype(float)

# Calcula los indicadores técnicos
data['momentum'] = ta.momentum.roc(data['close'], fillna=True)
data['ATR'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'], fillna=True)
data['ADX'] = ta.trend.adx(data['high'], data['low'], data['close'], fillna=True)
data['MACD'] = ta.trend.macd_diff(data['close'], fillna=True)

# Normaliza los datos y los indicadores técnicos
for column in ['open', 'high', 'low', 'close', 'momentum', 'ATR', 'ADX', 'MACD']:
    data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

# Procesar los datos para la entrada del modelo
data['next_close'] = data['close'].shift(-1)
data.dropna(inplace=True)

X = []
y = []

n = 48  # Número de pasos de tiempo en la secuencia

# Añade los indicadores técnicos a la entrada del modelo
for i in range(n, len(data)):
    X.append(data[['close', 'momentum', 'ATR', 'ADX', 'MACD']].iloc[i-n:i].values)
    y.append(data['next_close'].values[i])

# Convertir a numpy arrays
X, y = np.array(X), np.array(y)

# Redimensionar X a la forma adecuada para la entrada LSTM [muestras, pasos de tiempo, características]
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)
])

# Compilar el modelo
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=20, validation_data=(X_val, y_val))

# Guardar el modelo entrenado
model.save('modelos de entrenamiento/btc_price_prediction_model_Indicators_copy.h5')


/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
2023-05-16 10:13:11.805133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 10:13:11.805478: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registe

Epoch 1/200


2023-05-16 10:13:12.067813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-16 10:13:12.068917: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-16 10:13:12.069663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1289/1291 [============================>.] - ETA: 0s - loss: 6.6531e-04

2023-05-16 10:13:42.535720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-16 10:13:42.537038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-16 10:13:42.537960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1291/1291 [==============================] - 33s 24ms/step - loss: 6.6474e-04 - val_loss: 9.5213e-05
Epoch 2/200
1291/1291 [==============================] - 31s 24ms/step - loss: 9.2316e-05 - val_loss: 6.0981e-05
Epoch 3/200
1291/1291 [==============================] - 31s 24ms/step - loss: 8.6578e-05 - val_loss: 4.8134e-05
Epoch 4/200
1291/1291 [==============================] - 31s 24ms/step - loss: 7.3727e-05 - val_loss: 4.1089e-05
Epoch 5/200
1291/1291 [==============================] - 31s 24ms/step - loss: 7.2151e-05 - val_loss: 1.1244e-04
Epoch 6/200
1291/1291 [==============================] - 31s 24ms/step - loss: 5.9565e-05 - val_loss: 1.4295e-04
Epoch 7/200
1291/1291 [==============================] - 31s 24ms/step - loss: 5.9433e-05 - val_loss: 2.0830e-04
Epoch 8/200
1291/1291 [==============================] - 31s 24ms/step - loss: 5.2276e-05 - val_loss: 3.9984e-05
Epoch 9/200
1291/1291 [==============================] - 31s 24ms/step - loss: 4.8867e-05 - val_loss: 5.8077